In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
df.drop(columns=['ID', 'Unnamed: 0'], inplace=True)

In [4]:
X = df.drop(columns=['class'])
cols = X.columns
res = []
i = 0
ind = 0
for col in cols:
    if (i == 0):
        res = X[col]
        i = i + 1
    elif i == 5:
        res = pd.concat([res, X[col]])
        for j in range (ind - i, ind + 1):
            X[cols[j]] = X[cols[j]].transform(lambda x: x.fillna(res.mean(numeric_only = True)))
        i = 0
        res = pd.Series([])
    else:
        res = pd.concat([res, X[col]])
        i = i + 1
    ind = ind + 1
if i > 0:
    for j in range (ind - i, len(cols)):
        X[cols[j]] = X[cols[j]].transform(lambda x: x.fillna(res.mean(numeric_only = True)))

In [5]:
for col in cols:
    X[col] = X[col] / 10000
from sklearn.preprocessing import PolynomialFeatures
YX = X[cols].values
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(YX)
feature_names = poly.get_feature_names_out(cols)
df_poly = pd.DataFrame(X_poly, columns=feature_names)
X = pd.concat([X, df_poly], axis=1)

In [6]:
# for col in cols:
#     iq = X[col].describe()['75%'] - X[col].describe()['25%']
#     rval = X[col].describe()['75%'] + iq * 1.5
#     lval = X[col].describe()['25%'] - iq * 1.5
#     X[col][X[col] > rval] = rval
#     X[col][X[col] < lval] = lval

In [7]:
# missing_columns = [col for col in df.columns if df[col].isnull().any()]
# for col in missing_columns:
#     df[col] = df.groupby('class')[col].transform(lambda x: x.fillna(x.median()))
# df.fillna(df.median(numeric_only=True), inplace=True)
# df
X.isnull().sum()

20150720_N               0
20150602_N               0
20150517_N               0
20150501_N               0
20150415_N               0
                        ..
20140202_N 20140117_N    0
20140202_N 20140101_N    0
20140117_N^2             0
20140117_N 20140101_N    0
20140101_N^2             0
Length: 432, dtype: int64

In [8]:
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(X)
transformer.transform(X)

array([[-2.09495065, -0.87076466, -1.20205606, ..., -0.24934569,
        -0.45196499, -0.16594595],
       [-2.09610723, -0.88478631, -1.15520965, ..., -0.2170529 ,
        -0.47399117, -0.16009468],
       [-2.29474948, -1.35809547,  0.        , ...,  0.6959505 ,
        -0.96254565, -0.05642269],
       ...,
       [-1.66124875, -0.69894005,  0.        , ...,  0.09352438,
        -0.05366637, -0.07323518],
       [-1.11999914, -0.73593943, -0.7823979 , ...,  0.6959505 ,
        -0.23202749, -0.16848356],
       [ 0.07809899, -0.65293796,  0.0716327 , ..., -0.23744212,
        -0.06374935,  0.21939282]])

In [9]:
X

,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,20150226_N,20150210_N,20150125_N,...,20140218_N^2,20140218_N 20140202_N,20140218_N 20140117_N,20140218_N 20140101_N,20140202_N^2,20140202_N 20140117_N,20140202_N 20140101_N,20140117_N^2,20140117_N 20140101_N,20140101_N^2
0,0.063759,0.065867,-0.188203,-0.192436,0.099790,-0.173999,0.063009,0.387855,-0.132564,-0.094408,...,0.000447,-0.004656,-0.002494,0.000917,0.048533,0.026000,-0.009559,0.013928,-0.005121,0.001883
1,0.063424,0.059371,-0.162579,-0.167232,0.091420,-0.069239,0.070763,-0.167059,-0.140864,-0.098929,...,0.000488,-0.004970,-0.003005,0.001158,0.050625,0.030613,-0.011792,0.018511,-0.007130,0.002747
2,0.005802,-0.159916,0.469296,-0.105263,0.469296,-0.156463,0.387855,0.072979,-0.316214,0.387855,...,0.001363,-0.008131,0.014208,-0.004961,0.048493,-0.084740,0.029587,0.148078,-0.051701,0.018051
3,0.007252,0.469296,0.038044,-0.125693,0.051580,-0.141318,-0.080294,0.068325,-0.282940,-0.126754,...,0.000890,-0.006555,0.011480,-0.002466,0.048284,-0.084556,0.018166,0.148078,-0.031813,0.006835
4,0.113644,0.469296,0.469296,0.164783,0.193580,0.469296,0.215898,0.387855,0.124287,-0.264605,...,0.027578,-0.001940,-0.009433,-0.022537,0.000136,0.000663,0.001585,0.003227,0.007709,0.018418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.120770,0.098462,0.469296,0.116625,0.093748,0.107270,0.082390,0.094370,0.094462,0.104099,...,0.001317,0.003555,0.013964,0.001574,0.009600,0.037703,0.004249,0.148078,0.016688,0.001881
7996,0.217035,0.141972,0.136100,0.147871,0.098391,0.126211,0.142286,0.140197,0.137268,0.114234,...,0.001996,0.003448,0.007099,0.002265,0.005956,0.012264,0.003912,0.025251,0.008056,0.002570
7997,0.189568,0.145474,0.469296,0.103356,0.193038,0.105715,0.147160,0.387855,0.269363,0.237755,...,0.011990,0.019914,0.027394,0.013663,0.033075,0.045498,0.022693,0.062586,0.031216,0.015569
7998,0.346574,0.128332,0.041341,0.439105,0.114682,0.447305,0.161475,0.077047,0.387855,0.267600,...,0.077842,0.104896,0.107362,0.010835,0.141354,0.144677,0.014601,0.148078,0.014944,0.001508


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# model = LogisticRegression(
#     multi_class='multinomial',
#     solver='lbfgs',
#     max_iter=40,
#     class_weight='balanced'
# )

In [11]:
params = [
    {
        "penalty": ['l1', 'l2', 'elasticnet', None]
    },
    {
        "tol": [0.1, 0.001, 0.0001, 0.00001]
    },
    {
        "C": [1, 1.1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4]
    },
    {
        "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    },
    {
        "max_iter": [10, 100, 150, 200, 500, 1000]
    },
    {
        "class_weight": ['balanced']
    },
    {
        "multi_class": ['multinomial']
    }
]

In [12]:
model1 = LogisticRegression()

grid_logisticreg = GridSearchCV(model1, params, cv=5, scoring='accuracy', n_jobs=-1)
grid_logisticreg.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'penalty': ['l1', 'l2', 'elasticnet', None]},
                         {'tol': [0.1, 0.001, 0.0001, 1e-05]},
                         {'C': [1, 1.1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4]},
                         {'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                     'newton-cholesky', 'sag', 'saga']},
                         {'max_iter': [10, 100, 150, 200, 500, 1000]},
                         {'class_weight': ['balanced']},
                         {'multi_class': ['multinomial']}],
             scoring='accuracy')

In [13]:
y_pred = grid_logisticreg.predict(X_test)

print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))

              precision    recall  f1-score   support

        farm       0.86      0.76      0.81       168
      forest       0.96      0.99      0.98      1232
       grass       0.96      0.56      0.71        39
  impervious       0.81      0.87      0.83       134
     orchard       0.67      0.33      0.44         6
       water       0.83      0.48      0.61        21

    accuracy                           0.94      1600
   macro avg       0.85      0.66      0.73      1600
weighted avg       0.94      0.94      0.93      1600



In [14]:
test_data = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
test_data.shape

(2845, 29)

In [15]:
ID=test_data['ID']
test_data.drop(['ID', 'Unnamed: 0'],axis=1,inplace=True)
for col in cols:
    test_data[col] = test_data[col] / 10000
YX = test_data[cols].values
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(YX)
feature_names = poly.get_feature_names_out(cols)
df_poly = pd.DataFrame(X_poly, columns=feature_names)
test_data = pd.concat([test_data, df_poly], axis=1)
y_test = grid_logisticreg.predict(test_data)
y_test

array([0, 4, 4, ..., 5, 5, 3])

In [16]:
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

array(['farm', 'orchard', 'orchard', ..., 'water', 'water', 'impervious'],
      dtype=object)

In [17]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})
result

,ID,class
0,1,farm
1,2,orchard
2,3,orchard
3,4,farm
4,5,orchard
...,...,...
2840,2841,water
2841,2842,water
2842,2843,water
2843,2844,water


In [18]:
result.to_csv("submission.csv", index=False)